<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/ACL-cleanup/EEC_with_CN(w2v%2C_GloVe%2C_Fasttext).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load EEC

In [1]:
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip

--2019-03-15 01:33:19--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip.1’

Equity-Evaluation-C 100%[===================>] 100.16K  --.-KB/s    in 0.1s    

2019-03-15 01:33:19 (932 KB/s) - ‘Equity-Evaluation-Corpus.zip.1’ saved [102568/102568]



In [2]:
!unzip Equity-Evaluation-Corpus.zip

Archive:  Equity-Evaluation-Corpus.zip
replace Equity-Evaluation-Corpus.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Equity-Evaluation-Corpus.csv  
  inflating: README.txt              


In [3]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)

In [5]:
EEC[0:5]


,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


# Load GloVe embeddings

In [0]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

In [7]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /content/gensim_glove.840B.300d.txt.bin
2.65GB [00:47, 55.6MB/s]


In [8]:
glove = KeyedVectors.load_word2vec_format('/content/' + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

The glove embedding has been loaded!


# Load Word2vec embeddings


In [9]:
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:21, 76.5MB/s]


In [10]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

gzip: /content/GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)? y


In [11]:
word2vec = KeyedVectors.load_word2vec_format('/content/' + 'GoogleNews-vectors-negative300.bin', binary=True)
print('The word2vec embedding has been loaded!')

The word2vec embedding has been loaded!


# Load fasttext

In [12]:
!gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh

Downloading...
From: https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh
To: /content/fasttext.bin
2.42GB [00:58, 41.5MB/s]


In [13]:
fasttext = KeyedVectors.load_word2vec_format('/content/' + 'fasttext.bin', binary=True)
print('The fasttext embedding has been loaded!')

The fasttext embedding has been loaded!


# Load gender, racial name lists


In [14]:
## Gender name lists
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-03-15 02:47:11--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt.1’

male.txt.1          100%[===================>]  19.99K  --.-KB/s    in 0s      

2019-03-15 02:47:12 (146 MB/s) - ‘male.txt.1’ saved [20466/20466]

--2019-03-15 02:47:13--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt.1’

female.txt.1        100%[===================>]  34.91K  --.-KB/s    in 0.04s   

2019-03-15 02:47:13 (970 KB/s) - ‘female.txt.1’ saved [35751/35751]



In [15]:
## Racial names lists
!wget https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
!wget https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
!wget https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv
!wget https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv

--2019-03-15 02:47:14--  https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921600 (900K) [text/plain]
Saving to: ‘Black-Male-Names.csv.1’

Black-Male-Names.cs 100%[===================>] 900.00K  --.-KB/s    in 0.07s   

2019-03-15 02:47:14 (11.8 MB/s) - ‘Black-Male-Names.csv.1’ saved [921600/921600]

--2019-03-15 02:47:15--  https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.

# Get frequency list (used in calculating sentence embeddings)

In [16]:
!git clone https://github.com/PrincetonML/SIF

fatal: destination path 'SIF' already exists and is not an empty directory.


In [0]:
wikiWordsPath = '/content' + '/SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

frequencies = {}
with open(wikiWordsPath, "r+") as f_in:
    for line in f_in:
        frequencies[line.split(' ')[0]] = float(line.split(' ')[1])

# Load white/black names
2400 names each side

In [0]:
# This is for male names, don't run it when testing on female tasks
word_list_male = []

In [0]:

f = open('/content/White-Male-Names.csv', 'r+')
f.readline()
i=0
for line in f:
  i +=1
  word_list_male.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()
f = open('/content/Black-Male-Names.csv', 'r+')
i=0
f.readline()
for line in f:
  i+=1
  word_list_male.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()

In [20]:
len(word_list_male)

4800

In [0]:
word_list_female = []

In [0]:
f = open('/content/White-Female-Names.csv', 'r+')
f.readline()
i =0 
for line in f:
  i+=1
  word_list_female.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()
f = open('/content/Black-Female-Names.csv', 'r+')
f.readline()
i = 0
for line in f:
  i+=1
  word_list_female.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()

In [23]:
len(word_list_female)

4800

# Load gender names(Not used in Racial Debiasing)

In [0]:
#load names from larger name dataset
f = open('/content/male.txt', 'r+')
i = 0
word_list= []
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [26]:
len(word_list)

7944

# Experiemments

In [0]:
import nltk

class Sentence:
    
    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]

In [0]:
#Raw Sentence Embeddings
# calculate similarity scores on raw embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_raw(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])
    return proj1, proj2

In [0]:
#CN Sentence Embeddings
#cn_embeddings is a glocal variable
# calculate similarity scores on CN embeddings
def run_conceptor(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha = 1): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    sentiment_emb = negC.dot(model[sentiment].T).T
    proj1 = embedding1.dot(sentiment_emb)/norm(sentiment_emb)
    proj2 = embedding2.dot(sentiment_emb)/norm(sentiment_emb)
    return proj1, proj2

In [0]:
#Use all male names to find conceptor using word2vec
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list_male if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [31]:
cn_embeddings.shape

(3657, 300)

In [0]:
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001, 1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])

In [33]:
print('RAW vs CN data using WORD2VEC, male 4800 names')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


RAW vs CN data using WORD2VEC, male 4800 names
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          19.359635  3.104938e-57       12.406725  1.566748e-29
fear           21.366772  2.309479e-65       13.206601  1.415429e-32
joy            11.707836  6.245311e-27        4.849662  1.866064e-06
sadness        17.006111  1.142249e-47        8.030925  1.499917e-14


In [0]:
#Use all female names to find conceptor using word2vec
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list_female if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001, 1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [36]:
print('RAW vs CN data using WORD2VEC, female 4800 names')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, female 4800 names
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          13.849456  4.572745e-35        7.675373  1.669784e-13
fear            8.261900  3.016258e-15        0.190049  8.493809e-01
joy             5.400960  1.228679e-07        6.000689  4.920373e-09
sadness        19.543729  5.545095e-58       18.106588  3.877847e-52


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list_male if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p =[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='male')]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'glove', frequencies, 0.0001, 1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [39]:
print('RAW vs CN data using GLOVE, male 4800 names')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using GLOVE, male 4800 names
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -39.229215  5.467233e-130      -24.874147   2.729677e-79
fear          -53.682502  1.007069e-170      -43.764183  8.979959e-144
joy           -36.353356  1.032966e-120      -28.840793   2.160724e-94
sadness       -22.533354   4.817098e-70      -21.893513   1.758025e-67


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list_female if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='female')]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'glove', frequencies, 0.0001,1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [42]:
print('RAW vs CN data using GLOVE, female 4800 names')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using GLOVE, female 4800 names
         raw_t_statistic    raw_p_value  cn_t_statistic    cn_p_value
anger         -25.594525   4.332972e-82      -22.129255  1.992961e-68
fear          -33.008122  2.269988e-109      -29.898306  2.784183e-98
joy           -29.708214   1.378851e-97      -24.183388  1.386411e-76
sadness       -15.018152   1.128162e-39      -16.582704  5.910297e-46


In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list_male if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='male')]
  #print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001, 1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [45]:
print('RAW vs CN data using FASTTEXT, male 4800 names')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using FASTTEXT, male 4800 names
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -24.943738   1.461045e-79      -20.144836   2.012982e-60
fear          -38.831695  9.875491e-129      -36.201127  3.289849e-120
joy           -23.820355   3.732282e-75      -24.174769   1.498940e-76
sadness       -19.571735   4.267023e-58      -17.292249   7.895303e-49


In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list_female if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='female')]
  #print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001,1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [48]:
print('RAW vs CN data using FASTTEXT, female 4800 names')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using FASTTEXT, female 4800 names
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger           9.036385  1.130757e-17        4.293546  2.281516e-05
fear            3.073295  2.283697e-03       -4.045877  6.421761e-05
joy           -16.240469  1.423592e-44       -6.334529  7.338682e-10
sadness         4.786356  2.512300e-06        0.400246  6.892201e-01


# Using averaged sentiment embeddings

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm
# calculate similarity scores on raw embeddings
def run_raw(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    #average sentiment embedding based on several emotional words of the same sentiment
    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis=0)    
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(sen_embeddings)/norm(sen_embeddings)
    proj2 = embedding2.dot(sen_embeddings)/norm(sen_embeddings)


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
# calculate similarity scores on CN embeddings
def run_conceptor(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha =1): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis = 0)   
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    sen_embeddings = negC.dot(sen_embeddings.T).T
    proj1 = embedding1.dot(sen_embeddings)/norm(sen_embeddings)
    proj2 = embedding2.dot(sen_embeddings)/norm(sen_embeddings)
    return proj1, proj2

In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list_male if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001) 
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001, 1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [53]:
print('RAW vs CN data using WORD2VEC, male 4800 names, averaged sentimnent vector')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male 4800 names, averaged sentimnent vector
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          18.876432  2.863949e-55       13.611612  3.853190e-34
fear           18.281446  7.540234e-53       11.406851  7.872486e-26
joy            14.240858  1.340659e-36       10.060823  4.472136e-21
sadness        15.365000  4.667011e-41       11.866843  1.618999e-27


In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list_female if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p =[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001,1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [56]:
print('RAW vs CN data using WORD2VEC, female 4800 names, averaged sentiment emb')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, female 4800 names, averaged sentiment emb
         raw_t_statistic   raw_p_value  cn_t_statistic  cn_p_value
anger           6.118688  2.533850e-09        2.949185    0.003401
fear            4.115710  4.820979e-05        0.476732    0.633851
joy             1.940412  5.313473e-02        2.220383    0.027035
sadness         2.070742  3.911770e-02        1.391250    0.165036


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list_male if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'glove', frequencies, 0.0001,1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [59]:
print('RAW vs CN data using GLOVE, male 4800 names, averaged sentiment vector')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using GLOVE, male 4800 names, averaged sentiment vector
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -33.357260  1.393124e-110      -38.749684  1.798226e-128
fear          -41.600778  2.538149e-137      -40.133508  8.101200e-133
joy           -48.753785  7.681814e-158      -38.839614  9.320546e-129
sadness       -40.561857  3.825451e-134      -38.350224  3.369603e-127


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list_female if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'glove', frequencies, 0.0001,1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [62]:
print('RAW vs CN data using GLOVE, female 4800 names, average sentiment emb')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using GLOVE, female 4800 names, average sentiment emb
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -24.210266   1.087004e-76      -29.729904   1.148516e-97
fear          -33.929640  1.483594e-112      -35.047935  2.335524e-116
joy           -30.262349   1.316721e-99      -32.714851  2.394678e-108
sadness       -33.264375  2.922746e-110      -31.740498  6.492887e-105


In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list_male if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p =[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001, 1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [65]:
print('RAW vs CN data using FASTTEXT, male, 4k names, averaged sentiment emb')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using FASTTEXT, male, 4k names, averaged sentiment emb
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -46.225740  7.524439e-151      -44.888416  4.866403e-147
fear          -40.925205  2.918379e-135      -39.645938  2.684539e-131
joy           -43.957295  2.444131e-144      -39.509312  7.195105e-131
sadness       -44.227992  3.970227e-145      -39.335051  2.538114e-130


In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list_female if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p=[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_raw(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001,1)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [68]:
print('RAW vs CN data using FASTTEXT, female 4800 names, averaged sentiment emb')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using FASTTEXT, female 4800 names, averaged sentiment emb
         raw_t_statistic   raw_p_value  cn_t_statistic  cn_p_value
anger          12.255394  5.795978e-29        2.364344    0.018609
fear            7.172205  4.445886e-12       -1.922682    0.055334
joy            -0.056098  9.552957e-01       -2.137452    0.033256
sadness         5.408571  1.181476e-07       -2.410683    0.016439
